In [131]:
import requests as r
import sqlite3
import lxml
from bs4 import BeautifulSoup
import math
import time
import pprint
import threading

In [132]:
# index
index = "https://www.518.com.tw/job-index-P-1.html?i=1&am=1&ab=2032001,2032002,"
#        https://www.518.com.tw/job-index-P-1.html?i=1&am=1&ab=2032001,2032002,

In [141]:
# create table & drop table
with sqlite3.connect('job.sqlite') as conn:
    c = conn.cursor()
    try:
        c.execute('drop table job518')
        print('drop table job518')
    except:
        c.execute('''
            CREATE TABLE job518(
              jobID INTEGER PRIMARY KEY AUTOINCREMENT,
              title TEXT NOT NULL,
              href TEXT NOT NULL,
              times INTEGER,
              info TEXT)''')
        print('create table job518')

create table job518


In [135]:
# number of total job 
def getPage1111(href):
#     import locale
#     locale.setlocale(locale.LC_ALL, 'en_US.UTF-8') 
    res = r.get(href)
    
    return int(BeautifulSoup(res.text, 'lxml').select('span.pagecountnum')[0].text.split('頁')[0].split('/')[1].strip())

totalPages = getPage1111(index)
totalPages

54

In [136]:
res = r.get(index)
soup = BeautifulSoup(res.text, 'lxml').select('div#listContent')

In [137]:
# test program -> get href
soup[0].select('a')[0]['href'][0:]

'https://www.518.com.tw/Android程式設計師-新北市-三重區-job-886709.html?kw=&pi=1'

In [138]:
# sqlite insert&update -> job518
def insert_href(title, href):
    try:
        with sqlite3.connect('job.sqlite') as conn:

            # Query histry, whether we have insert it before...
            c = conn.cursor()
            qryString = "SELECT href FROM job518 where href=:href"
            c.execute(qryString, {'href' : href})
            # if there are nothing like href
            if len(c.fetchall()) == 0:
                # insert new one
                insert_string = "INSERT INTO job518 (title, href, times) VALUES (?, ?, 1)"
                c.execute(insert_string, (title, href))
            else:
                update_string = "UPDATE job518 SET times = times + 1 WHERE href = ?"
                c.execute(update_string, (href,))

    except ConnectionError as e:
        print(e)
        print(href)

In [142]:
# main program
for page in range(1, totalPages + 1):
# for page in range(1, 2):
    href = "https://www.518.com.tw/job-index-P-{}.html?i=1&am=1&ab=2032001,2032002,".format(page)
    soup = BeautifulSoup(r.get(href).text, 'lxml')
    jidSoup = soup.select('div#listContent > ul')
    totalJid = len(jidSoup)
    print(totalJid)
    print('page: ' + str(page))
    for jid in range(0, totalJid-3):
        title = jidSoup[jid].select('a')[0]['title'][0:].split('職缺名：')[1].split('\n')[0]
        href = jidSoup[jid].select('a')[0]['href'][0:].replace(' ','')
        print(title)
        print(href)
        insert_href(title, href)
    for jid in range(totalJid-3, totalJid):
        title = jidSoup[jid].select('a')[0]['title'][0:]
        href = jidSoup[jid].select('a')[0]['href'][0:].replace(' ','')
        print(title)
        print(href)
        insert_href(title, href)
    
#     time.sleep(2)
res.close()
print('done!!')

26
page: 1
Android程式設計師
https://www.518.com.tw/Android程式設計師-新北市-三重區-job-886709.html?kw=&pi=1
Internet程式設計師、PHP網站程式設計師
https://www.518.com.tw/Internet程式設計師、PHP網站程式設計師-新北市-三重區-job-949808.html?kw=&pi=1
101_網站工程師
https://www.518.com.tw/101_網站工程師-新北市-三重區-job-660380.html?kw=&pi=1
BIM研發經理
https://www.518.com.tw/BIM研發經理-台北市-信義區-job-1356565.html?kw=&pi=1
軟體研發工程師
https://www.518.com.tw/軟體研發工程師-台北市-信義區-job-1356524.html?kw=&pi=1
IT網管工程師
https://www.518.com.tw/IT網管工程師-台中市-西區-job-1348801.html?kw=&pi=1
電腦工程師
https://www.518.com.tw/電腦工程師-新北市-樹林區-job-829908.html?kw=&pi=1
網頁美工印刷電腦設計師
https://www.518.com.tw/網頁美工印刷電腦設計師-桃園市-龍潭區-job-1163675.html?kw=&pi=1
系統分析專案主管(台北)
https://www.518.com.tw/系統分析專案主管(台北)-台北市-中正區-job-1255792.html?kw=&pi=1
程式設計師(台北)
https://www.518.com.tw/程式設計師(台北)-台北市-中正區-job-863111.html?kw=&pi=1
徵美工助理兼網拍內勤人員
https://www.518.com.tw/徵美工助理兼網拍內勤人員-台中市-大里區-job-208730.html?kw=&pi=1
電腦維修工程師
https://www.518.com.tw/電腦維修工程師-桃園市-桃園區-job-1009036.html?kw=&pi=1
相機網拍行政人員
https://www.518.com.tw/相機網拍行政人員-新竹縣

23
page: 6
資訊助理人員
https://www.518.com.tw/資訊助理人員-高雄市-左營區-job-602256.html?kw=&pi=6
會計出納
https://www.518.com.tw/會計出納-高雄市-左營區-job-1269408.html?kw=&pi=6
行政助理
https://www.518.com.tw/行政助理-台中市-北屯區-job-1342376.html?kw=&pi=6
韌體設計工程師
https://www.518.com.tw/韌體設計工程師-台南市-後壁區-job-525382.html?kw=&pi=6
MIS助理工程師
https://www.518.com.tw/MIS助理工程師-台北市-南港區-job-1328982.html?kw=&pi=6
約聘工程師助理-二個月
https://www.518.com.tw/約聘工程師助理-二個月-台北市-南港區-job-1328573.html?kw=&pi=6
包裝員/作業員
https://www.518.com.tw/包裝員作業員-台南市-後壁區-job-150956.html?kw=&pi=6
網站維護管理員
https://www.518.com.tw/網站維護管理員-台北市-內湖區-job-1222644.html?kw=&pi=6
程式設計助理工程師
https://www.518.com.tw/程式設計助理工程師-台中市-南區-job-1340901.html?kw=&pi=6
網頁設計師
https://www.518.com.tw/網頁設計師-高雄市-仁武區-job-1293901.html?kw=&pi=6
購物網站管理人員
https://www.518.com.tw/購物網站管理人員-高雄市-仁武區-job-986664.html?kw=&pi=6
[KSD]電腦維修工讀生/週休二日/晚上不會加班(路竹)
https://www.518.com.tw/[KSD]電腦維修工讀生週休二日晚上不會加班(路竹)-高雄市-路竹區-job-1333813.html?kw=&pi=6
世大運 資通處設備組 行政專員
https://www.518.com.tw/世大運資通處設備組行政專員-台北市-松山區-job-1348119.html?kw=

23
page: 11
內勤-行政,企劃,資訊人員
https://www.518.com.tw/內勤-行政,企劃,資訊人員-台北市-大安區-job-1275394.html?kw=&pi=11
網站工程師
https://www.518.com.tw/網站工程師-桃園市-中壢區-job-1329230.html?kw=&pi=11
程式設計工程師
https://www.518.com.tw/程式設計工程師-高雄市-岡山區-job-1356311.html?kw=&pi=11
網站人員
https://www.518.com.tw/網站人員-台北市-松山區-job-1021998.html?kw=&pi=11
Android 軟體工程師
https://www.518.com.tw/Android軟體工程師-台北市-中山區-job-1166558.html?kw=&pi=11
資訊助理人員
https://www.518.com.tw/資訊助理人員-台中市-豐原區-job-1138114.html?kw=&pi=11
電控電機系統工程師
https://www.518.com.tw/電控電機系統工程師-高雄市-岡山區-job-336284.html?kw=&pi=11
網站後端工程師(Java)
https://www.518.com.tw/網站後端工程師(Java)-台北市-內湖區-job-1328691.html?kw=&pi=11
Android App開發工程師
https://www.518.com.tw/AndroidApp開發工程師-台北市-內湖區-job-1293108.html?kw=&pi=11
iOS App開發工程師
https://www.518.com.tw/iOSApp開發工程師-台北市-內湖區-job-1293106.html?kw=&pi=11
機房運維工程師
https://www.518.com.tw/機房運維工程師-台北市-內湖區-job-1293104.html?kw=&pi=11
EDM軟體設計工程師
https://www.518.com.tw/EDM軟體設計工程師-台北市-內湖區-job-1293099.html?kw=&pi=11
遊戲開發工程師
https://www.518.com.tw/遊戲開發工程師-台北市

23
page: 16
資訊部專員[學生實習](兼職)
https://www.518.com.tw/資訊部專員[學生實習](兼職)-台北市-大安區-job-1236596.html?kw=&pi=16
資訊部專員[學生實習]
https://www.518.com.tw/資訊部專員[學生實習]-台北市-大安區-job-1236594.html?kw=&pi=16
資訊部助理(兼職)
https://www.518.com.tw/資訊部助理(兼職)-台北市-大安區-job-1236657.html?kw=&pi=16
資訊部助理[學生實習]
https://www.518.com.tw/資訊部助理[學生實習]-台北市-大安區-job-1236658.html?kw=&pi=16
資訊部助理[學生實習](兼職)
https://www.518.com.tw/資訊部助理[學生實習](兼職)-台北市-大安區-job-1236661.html?kw=&pi=16
網頁設計師(兼職)
https://www.518.com.tw/網頁設計師(兼職)-台北市-大安區-job-1236675.html?kw=&pi=16
網頁設計師[學生實習]
https://www.518.com.tw/網頁設計師[學生實習]-台北市-大安區-job-1236678.html?kw=&pi=16
網頁設計師[學生實習](兼職)
https://www.518.com.tw/網頁設計師[學生實習](兼職)-台北市-大安區-job-1236679.html?kw=&pi=16
程式設計師
https://www.518.com.tw/程式設計師-台北市-大安區-job-1236714.html?kw=&pi=16
資訊部助理
https://www.518.com.tw/資訊部助理-台北市-大安區-job-1236653.html?kw=&pi=16
網頁設計師
https://www.518.com.tw/網頁設計師-台北市-大安區-job-250169.html?kw=&pi=16
網站商品上架人員
https://www.518.com.tw/網站商品上架人員-台北市-中正區-job-1172298.html?kw=&pi=16
資料處理人員
https://www.518.com.tw/資

23
page: 21
資料庫管理員
https://www.518.com.tw/資料庫管理員-台北市-內湖區-job-1213997.html?kw=&pi=21
APP應用程式工程師
https://www.518.com.tw/APP應用程式工程師-台北市-內湖區-job-1213983.html?kw=&pi=21
技術主管
https://www.518.com.tw/技術主管-台北市-中山區-job-1166619.html?kw=&pi=21
系統/網路工程師
https://www.518.com.tw/系統網路工程師-台北市-內湖區-job-1213981.html?kw=&pi=21
網頁程式工程師
https://www.518.com.tw/網頁程式工程師-台北市-內湖區-job-1213974.html?kw=&pi=21
軟體工程師
https://www.518.com.tw/軟體工程師-台中市-大里區-job-1304747.html?kw=&pi=21
網路管理工程師
https://www.518.com.tw/網路管理工程師-台中市-南屯區-job-1280939.html?kw=&pi=21
網路行銷專員
https://www.518.com.tw/網路行銷專員-桃園市-全區-job-804193.html?kw=&pi=21
資訊人員
https://www.518.com.tw/資訊人員-台中市-西屯區-job-1190832.html?kw=&pi=21
網站行銷企劃
https://www.518.com.tw/網站行銷企劃-台中市-北屯區-job-740003.html?kw=&pi=21
電子商務平台後台JAVA開發工程師
https://www.518.com.tw/電子商務平台後台JAVA開發工程師-新竹縣-竹北市-job-1151048.html?kw=&pi=21
韌體工程師
https://www.518.com.tw/韌體工程師-新北市-三重區-job-386526.html?kw=&pi=21
弱電工程人員 - 無經驗可(補助住宿津貼)
https://www.518.com.tw/弱電工程人員-無經驗可(補助住宿津貼)-台中市-西區-job-1325562.html?kw=&pi=21
電子工程

...

23
page: 31
網路電腦工程師
https://www.518.com.tw/網路電腦工程師-新北市-林口區-job-766277.html?kw=&pi=31
資訊幹部(國光店)
https://www.518.com.tw/資訊幹部(國光店)-台中市-大里區-job-854606.html?kw=&pi=31
技術部_Android資深軟體工程師
https://www.518.com.tw/技術部_Android資深軟體工程師-台北市-松山區-job-1167358.html?kw=&pi=31
PHP網頁與程式設計工程師
https://www.518.com.tw/PHP網頁與程式設計工程師-台北市-萬華區-job-903662.html?kw=&pi=31
資深網頁視覺設計師(UI Designer)(週休二日)
https://www.518.com.tw/資深網頁視覺設計師(UIDesigner)(週休二日)-台中市-全區-job-913699.html?kw=&pi=31
(SUM世和汽車)汽車美容、網管
https://www.518.com.tw/(SUM世和汽車)汽車美容、網管-彰化縣-田中鎮-job-1213384.html?kw=&pi=31
網頁設計師/美術設計
https://www.518.com.tw/網頁設計師美術設計-台北市-中山區-job-1166550.html?kw=&pi=31
Internet程式設計師
https://www.518.com.tw/Internet程式設計師-台中市-太平區-job-961060.html?kw=&pi=31
電腦軟體設計工程師
https://www.518.com.tw/電腦軟體設計工程師-高雄市-苓雅區-job-454078.html?kw=&pi=31
網路行銷與網路管理
https://www.518.com.tw/網路行銷與網路管理-台北市-松山區-job-1059957.html?kw=&pi=31
資訊人員
https://www.518.com.tw/資訊人員-台北市-內湖區-job-1161221.html?kw=&pi=31
網站前端工程師
https://www.518.com.tw/網站前端工程師-新竹市-香山區-job-1092993.html?k

23
page: 36
程式開發專員
https://www.518.com.tw/程式開發專員-新北市-樹林區-job-892380.html?kw=&pi=36
SAP工程師
https://www.518.com.tw/SAP工程師-桃園市-觀音區-job-851093.html?kw=&pi=36
軟體工程師
https://www.518.com.tw/軟體工程師-新北市-樹林區-job-717615.html?kw=&pi=36
資訊人員
https://www.518.com.tw/資訊人員-新北市-板橋區-job-1217521.html?kw=&pi=36
桃園區業務工程
https://www.518.com.tw/桃園區業務工程-桃園市-八德區-job-383014.html?kw=&pi=36
Android/IOS APP程式設計師
https://www.518.com.tw/AndroidIOSAPP程式設計師-台北市-南港區-job-1109768.html?kw=&pi=36
APP程式設計師
https://www.518.com.tw/APP程式設計師-台北市-大同區-job-1097205.html?kw=&pi=36
ASP.NET開發工程師
https://www.518.com.tw/ASP.NET開發工程師-台北市-內湖區-job-839889.html?kw=&pi=36
APP 開發人員
https://www.518.com.tw/APP開發人員-桃園市-桃園區-job-1059869.html?kw=&pi=36
前端工程師
https://www.518.com.tw/前端工程師-高雄市-苓雅區-job-990324.html?kw=&pi=36
軟韌體工程師
https://www.518.com.tw/軟韌體工程師-新北市-汐止區-job-281899.html?kw=&pi=36
PHP網站程式設計師
https://www.518.com.tw/PHP網站程式設計師-台北市-松山區-job-792262.html?kw=&pi=36
網路程式開發工程師
https://www.518.com.tw/網路程式開發工程師-台中市-西區-job-893319.html?kw=&pi=36
網頁工程師(PHP

23
page: 41
網路管理工程師
https://www.518.com.tw/網路管理工程師-台北市-士林區-job-768282.html?kw=&pi=41
Internet .Net 網頁工程師
https://www.518.com.tw/Internet.Net網頁工程師-台北市-中正區-job-959699.html?kw=&pi=41
Windows AP工程師
https://www.518.com.tw/WindowsAP工程師-台中市-南屯區-job-671951.html?kw=&pi=41
Flash遊戲開發工程師
https://www.518.com.tw/Flash遊戲開發工程師-台中市-南屯區-job-671915.html?kw=&pi=41
PHP 軟體工程師
https://www.518.com.tw/PHP軟體工程師-台中市-南屯區-job-671931.html?kw=&pi=41
資訊人員
https://www.518.com.tw/資訊人員-台中市-清水區-job-1028938.html?kw=&pi=41
軟體工程師
https://www.518.com.tw/軟體工程師-新北市-新店區-job-596217.html?kw=&pi=41
軔體研發工程師
https://www.518.com.tw/軔體研發工程師-新北市-新店區-job-659181.html?kw=&pi=41
影像處理工程師
https://www.518.com.tw/影像處理工程師-新北市-新店區-job-659179.html?kw=&pi=41
軟體客服人員
https://www.518.com.tw/軟體客服人員-台中市-東區-job-532027.html?kw=&pi=41
網頁設計師 (工程師) / PHP Developer
https://www.518.com.tw/網頁設計師(工程師)PHPDeveloper-台北市-信義區-job-923501.html?kw=&pi=41
網頁/手機程式開發工程師(APP/Web)
https://www.518.com.tw/網頁手機程式開發工程師(APPWeb)-台北市-南港區-job-856862.html?kw=&pi=41
設備技術工程主管
https://

23
page: 46
程式資訊助理人員
https://www.518.com.tw/程式資訊助理人員-台北市-北投區-job-1137005.html?kw=&pi=46
Unity3D 軟體工程師
https://www.518.com.tw/Unity3D軟體工程師-台北市-大安區-job-1174728.html?kw=&pi=46
遊戲程式工程師(Server)
https://www.518.com.tw/遊戲程式工程師(Server)-台北市-信義區-job-1009967.html?kw=&pi=46
軟體工程師助理
https://www.518.com.tw/軟體工程師助理-彰化縣-員林市-job-1225757.html?kw=&pi=46
資深C++軟體工程師(Client)
https://www.518.com.tw/資深C++軟體工程師(Client)-台北市-信義區-job-1009953.html?kw=&pi=46
Senior C#.net Engineer
https://www.518.com.tw/SeniorC.netEngineer-新北市-土城區-job-1198440.html?kw=&pi=46
軟體設計助理工程師
https://www.518.com.tw/軟體設計助理工程師-台南市-安平區-job-1039804.html?kw=&pi=46
IT - 網頁設計師
https://www.518.com.tw/IT-網頁設計師-台北市-信義區-job-1075509.html?kw=&pi=46
網路程式設計師
https://www.518.com.tw/網路程式設計師-新北市-新莊區-job-747302.html?kw=&pi=46
遊戲軟體工程師
https://www.518.com.tw/遊戲軟體工程師-高雄市-前鎮區-job-956683.html?kw=&pi=46
工程
https://www.518.com.tw/工程-台北市-全區-job-814288.html?kw=&pi=46
軟體測試工程師
https://www.518.com.tw/軟體測試工程師-新北市-新莊區-job-862488.html?kw=&pi=46
全端工程師 (Full Stack)
https://ww

20
page: 51
軟體業務/行銷人員
https://www.518.com.tw/軟體業務行銷人員-桃園市-八德區-job-1029284.html?kw=&pi=51
網路應用與管理
https://www.518.com.tw/網路應用與管理-台中市-北屯區-job-1313288.html?kw=&pi=51
系統工程師
https://www.518.com.tw/系統工程師-台南市-北區-job-679367.html?kw=&pi=51
電腦工程師(後端)
https://www.518.com.tw/電腦工程師(後端)-台中市-南屯區-job-1191073.html?kw=&pi=51
資訊總務人員
https://www.518.com.tw/資訊總務人員-台北市-南港區-job-181458.html?kw=&pi=51
PHP工程師
https://www.518.com.tw/PHP工程師-台中市-西屯區-job-823283.html?kw=&pi=51
軟體工程師
https://www.518.com.tw/軟體工程師-桃園市-蘆竹區-job-1312935.html?kw=&pi=51
電腦維修工程師 _(全職)
https://www.518.com.tw/電腦維修工程師_(全職)-台中市-北屯區-job-64055.html?kw=&pi=51
105年研發替代役-網站程式設計師
https://www.518.com.tw/105年研發替代役-網站程式設計師-台北市-中山區-job-1119056.html?kw=&pi=51
Ruby on Rails(ROR)程式工程師
https://www.518.com.tw/RubyonRails(ROR)程式工程師-台中市-西區-job-1226581.html?kw=&pi=51
程式設計工程師
https://www.518.com.tw/程式設計工程師-台中市-沙鹿區-job-1297995.html?kw=&pi=51
兼職-APP程式設計師
https://www.518.com.tw/兼職-APP程式設計師-台北市-大同區-job-1097207.html?kw=&pi=51
機率工程師
https://www.518.com.tw/機率工程師-台中市-西區-j

In [143]:
res.close()

In [144]:
# Query result program
# 2017/05/08 : ? web(exclude repeat)
with sqlite3.connect('job.sqlite') as conn:
    c = conn.cursor()
    pprint.pprint(list(c.execute('select * from job518')))

[(1,
  'Android程式設計師',
  'https://www.518.com.tw/Android程式設計師-新北市-三重區-job-886709.html?kw=&pi=1',
  1,
  None),
 (2,
  'Internet程式設計師、PHP網站程式設計師',
  'https://www.518.com.tw/Internet程式設計師、PHP網站程式設計師-新北市-三重區-job-949808.html?kw=&pi=1',
  1,
  None),
 (3,
  '101_網站工程師',
  'https://www.518.com.tw/101_網站工程師-新北市-三重區-job-660380.html?kw=&pi=1',
  1,
  None),
 (4,
  'BIM研發經理',
  'https://www.518.com.tw/BIM研發經理-台北市-信義區-job-1356565.html?kw=&pi=1',
  1,
  None),
 (5,
  '軟體研發工程師',
  'https://www.518.com.tw/軟體研發工程師-台北市-信義區-job-1356524.html?kw=&pi=1',
  1,
  None),
 (6,
  'IT網管工程師',
  'https://www.518.com.tw/IT網管工程師-台中市-西區-job-1348801.html?kw=&pi=1',
  1,
  None),
 (7,
  '電腦工程師',
  'https://www.518.com.tw/電腦工程師-新北市-樹林區-job-829908.html?kw=&pi=1',
  1,
  None),
 (8,
  '網頁美工印刷電腦設計師',
  'https://www.518.com.tw/網頁美工印刷電腦設計師-桃園市-龍潭區-job-1163675.html?kw=&pi=1',
  1,
  None),
 (9,
  '系統分析專案主管(台北)',
  'https://www.518.com.tw/系統分析專案主管(台北)-台北市-中正區-job-1255792.html?kw=&pi=1',
  1,
  None),
 (10,
  '程式設計師(台北)',
  'h

 (175,
  '業務工程師',
  'https://www.518.com.tw/業務工程師-台北市-內湖區-job-1226440.html?kw=&pi=8',
  1,
  None),
 (176,
  'iOS APP 程式開發工程師',
  'https://www.518.com.tw/iOSAPP程式開發工程師-桃園市-平鎮區-job-662381.html?kw=&pi=8',
  1,
  None),
 (177,
  'RHCE 工程師',
  'https://www.518.com.tw/RHCE工程師-台北市-全區-job-964837.html?kw=&pi=8',
  1,
  None),
 (178,
  '安卓刷機人員',
  'https://www.518.com.tw/安卓刷機人員-高雄市-鳳山區-job-921760.html?kw=&pi=8',
  1,
  None),
 (179,
  '前端介面工程師',
  'https://www.518.com.tw/前端介面工程師-台北市-大同區-job-1223207.html?kw=&pi=8',
  1,
  None),
 (180,
  '程式設計師',
  'https://www.518.com.tw/程式設計師-新北市-中和區-job-1278352.html?kw=&pi=8',
  1,
  None),
 (181,
  '程式設計師',
  'https://www.518.com.tw/程式設計師-台中市-全區-job-1349318.html?kw=&pi=8',
  1,
  None),
 (182,
  'Internet程式設計師',
  'https://www.518.com.tw/Internet程式設計師-台北市-中山區-job-1333649.html?kw=&pi=8',
  1,
  None),
 (183,
  'SAP工程師',
  'https://www.518.com.tw/SAP工程師-台中市-潭子區-job-1254695.html?kw=&pi=8',
  1,
  None),
 (184,
  '數位機上盒工程師(基隆)',
  'https://www.518.com.tw/數位機上盒工程

 (744,
  '軟體工程師',
  'https://www.518.com.tw/軟體工程師-苗栗縣-三義鄉-job-828856.html?kw=&pi=33',
  1,
  None),
 (745,
  '網站程式設計師',
  'https://www.518.com.tw/網站程式設計師-台中市-豐原區-job-553732.html?kw=&pi=33',
  1,
  None),
 (746,
  '資訊助理',
  'https://www.518.com.tw/資訊助理-台中市-南屯區-job-789178.html?kw=&pi=33',
  1,
  None),
 (747,
  '軟體助理工程師',
  'https://www.518.com.tw/軟體助理工程師-新北市-三重區-job-639482.html?kw=&pi=33',
  1,
  None),
 (748,
  '有線電視裝機人員',
  'https://www.518.com.tw/有線電視裝機人員-台北市-全區-job-789922.html?kw=&pi=33',
  1,
  None),
 (749,
  '軟體工程師',
  'https://www.518.com.tw/軟體工程師-新北市-三重區-job-646107.html?kw=&pi=33',
  1,
  None),
 (750,
  'PHP程式設計師',
  'https://www.518.com.tw/PHP程式設計師-台北市-中山區-job-706461.html?kw=&pi=33',
  1,
  None),
 (751,
  'PLC軟體設計',
  'https://www.518.com.tw/PLC軟體設計-新北市-樹林區-job-696511.html?kw=&pi=33',
  1,
  None),
 (752,
  '程式設計師',
  'https://www.518.com.tw/程式設計師-台中市-南屯區-job-294644.html?kw=&pi=33',
  1,
  None),
 (753,
  '資訊人員',
  'https://www.518.com.tw/資訊人員-台中市-太平區-job-1240681.html?kw=&pi

In [145]:
# Query maximun times
with sqlite3.connect('job.sqlite') as conn:
    c = conn.cursor()
#     pprint.pprint(list(c.execute('select * from job1111 where times > 1')))
    pprint.pprint(list(c.execute('select count(times), sum(times) from job518 where times > 1')))

[(0, None)]
